# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [3]:
import myImageLib as mil
import os
import numpy as np

In [84]:
# Construct args_list 
args_list = []
local_input = r'D:\Wei\02042020'
remote_input = r'~\Data\Wei\02042020'
remote_output = r'~\Data\02182020\img\result'
nd2Dirs = mil.dirrec(local_input, '*.nd2')
for nd2Dir in nd2Dirs:
    folder, file = os.path.split(nd2Dir)
    name, ext = os.path.splitext(file)
    input_dir = os.path.join(remote_input)
    output_dir = os.path.join(remote_output)
    args = ' '.join((os.path.join(input_dir, file), '')).replace('\\', '/') ### Pay attention here
    args_list.append(args)
# test master_input = I:\Github\Python\generic_proc\test_images
# test master_output = I:\Github\Python\generic_proc\test_images\result

In [85]:
args_list

['~/Data/Wei/02042020/20-1.nd2 ',
 '~/Data/Wei/02042020/20-2.nd2 ',
 '~/Data/Wei/02042020/20-3.nd2 ',
 '~/Data/Wei/02042020/40-1.nd2 ',
 '~/Data/Wei/02042020/40-2.nd2 ',
 '~/Data/Wei/02042020/40-3.nd2 ',
 '~/Data/Wei/02042020/60-1.nd2 ',
 '~/Data/Wei/02042020/60-2.nd2 ',
 '~/Data/Wei/02042020/60-3.nd2 ',
 '~/Data/Wei/02042020/80-1.nd2 ',
 '~/Data/Wei/02042020/80-2.nd2 ',
 '~/Data/Wei/02042020/80-3.nd2 ']

In [79]:
# batch rename files
for nd2Dir in nd2Dirs:
    os.rename(nd2Dir, nd2Dir.replace('no', ''))

In [86]:
code_path = r'~/code/Python/generic_proc/py_files'
py_name = r'to_tif.py'
walltime = int(len(args_list) * 0.3)# hr
nodes = 1
memory = 1 # gb

In [87]:
print('#!/bin/bash -l')
print('#PBS -l walltime={0}:00:00,nodes={1}:ppn=8,mem={2}gb'.format(walltime, nodes, memory))
print('#PBS -m abe')
print('#PBS -M liux3141@umn.edu\n')
print('cd {}'.format(code_path))
print('module load python3')
print('source activate pythonEnv')

for args in args_list:
    print('python {} {} &'.format(py_name, args))
print('wait')

#!/bin/bash -l
#PBS -l walltime=3:00:00,nodes=1:ppn=8,mem=1gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/generic_proc/py_files
module load python3
source activate pythonEnv
python to_tif.py ~/Data/Wei/02042020/20-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-3.nd2  &
wait


In [2]:
# job file generator
c = [20, 40, 60, 80]
n = [1, 2, 3]
for cc in c:
    for nn in n:
        name = '{0}-{1}'.format(cc, nn)
        print('python dc_adv.py ~/Data/02042020/'+name+'/8-bit ~/Data/02042020/piv_result_50/'+name+' ~/Data/02042020/dc+adv/'+name+' 10 &')

python dc_adv.py ~/Data/02042020/20-1/8-bit ~/Data/02042020/piv_result_50/20-1 ~/Data/02042020/dc+adv/20-1 10 &
python dc_adv.py ~/Data/02042020/20-2/8-bit ~/Data/02042020/piv_result_50/20-2 ~/Data/02042020/dc+adv/20-2 10 &
python dc_adv.py ~/Data/02042020/20-3/8-bit ~/Data/02042020/piv_result_50/20-3 ~/Data/02042020/dc+adv/20-3 10 &
python dc_adv.py ~/Data/02042020/40-1/8-bit ~/Data/02042020/piv_result_50/40-1 ~/Data/02042020/dc+adv/40-1 10 &
python dc_adv.py ~/Data/02042020/40-2/8-bit ~/Data/02042020/piv_result_50/40-2 ~/Data/02042020/dc+adv/40-2 10 &
python dc_adv.py ~/Data/02042020/40-3/8-bit ~/Data/02042020/piv_result_50/40-3 ~/Data/02042020/dc+adv/40-3 10 &
python dc_adv.py ~/Data/02042020/60-1/8-bit ~/Data/02042020/piv_result_50/60-1 ~/Data/02042020/dc+adv/60-1 10 &
python dc_adv.py ~/Data/02042020/60-2/8-bit ~/Data/02042020/piv_result_50/60-2 ~/Data/02042020/dc+adv/60-2 10 &
python dc_adv.py ~/Data/02042020/60-3/8-bit ~/Data/02042020/piv_result_50/60-3 ~/Data/02042020/dc+adv/60

In [6]:
intervals = np.linspace(10, 990, 100)
for i in intervals:
    interval = int(i)
    print('python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt={0:d} {1:d} 10 25 &'.format(interval, interval))

python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=10 10 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=19 19 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=29 29 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=39 39 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=49 49 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=59 59 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=69 69 10 25 &
python dc_adv.py ~/Data/02042020/80-1/8-bit ~/Data/02042020/piv_result_50/80-1 ~/Data/02042020/dc_adv/01/dt=79 79 10 25 &
python dc_adv.py ~/Data/

In [2]:
numL = range(0, 8)
for num in numL:
    print('python df2_kinetics.py ~/Data/06162020/{0:02d}/8-bit ~/Data/06162020/df2_kinetics/{0:02d} 100 &'.format(num))

python df2_kinetics.py ~/Data/06162020/00/8-bit ~/Data/06162020/df2_kinetics/00 100 &
python df2_kinetics.py ~/Data/06162020/01/8-bit ~/Data/06162020/df2_kinetics/01 100 &
python df2_kinetics.py ~/Data/06162020/02/8-bit ~/Data/06162020/df2_kinetics/02 100 &
python df2_kinetics.py ~/Data/06162020/03/8-bit ~/Data/06162020/df2_kinetics/03 100 &
python df2_kinetics.py ~/Data/06162020/04/8-bit ~/Data/06162020/df2_kinetics/04 100 &
python df2_kinetics.py ~/Data/06162020/05/8-bit ~/Data/06162020/df2_kinetics/05 100 &
python df2_kinetics.py ~/Data/06162020/06/8-bit ~/Data/06162020/df2_kinetics/06 100 &
python df2_kinetics.py ~/Data/06162020/07/8-bit ~/Data/06162020/df2_kinetics/07 100 &
